In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Losujemy miasta do odwiedzenia

- **number_of_points**: liczba miast do odwiedzenia
- **max_value**: max wartosc wspolrzednych X i Y

In [2]:
def random_points(number_of_points, max_value):
    np.random.seed(14)
    x = np.random.uniform(0, max_value, number_of_points)
    y = np.random.uniform(0, max_value, number_of_points)
    cites_df = pd.DataFrame({'X': x, 'Y': y})
    cites_df.index.name = 'City number'
    return cites_df

In [3]:
cities_df = random_points(30, 200)


### Generujemy populację początkową
- **population_size**: rozmiar populacji
- **cities_df**: DataFrame ze wspolrzednymi miast 

In [4]:
def generate_population(population_size, cities_df):
    population = []
    for i in range(population_size):
        population.append(np.random.permutation(cities_df.index))
    population_df = pd.DataFrame(population)
    population_df.index.name = 'Numer osobnika'
    population_df.columns.names = ['Numer chromosomu']
    return population_df

In [5]:
population_df = generate_population(50, cities_df)


### Oceniamy populacje

In [6]:
def rate_population(population_df, cities_df):
        # ocena populacji
        #np.sum(np.sqrt((temp_df['X']**2 - temp_df['Y']**2)))
        distances = []
        for osobnik in range(len(population_df)):
            temp = population_df.loc[osobnik]
            temp2 = cities_df.loc[temp]
            distance = np.sum(np.sqrt(np.diff(temp2['X'])**2 + np.diff(temp2['Y'])**2))
            distances.append(distance)
        population_df['distance'] = distances
        return population_df.sort_values(by='distance', ascending=True)

In [7]:
rated_population = rate_population(population_df, cities_df)


**Wybieramy num_of_infividuals najlepszych osobników**

In [21]:
def selection(population_df, method, num_of_indiviudals):
    # while num of samples
    if method == 'simple':
            # choose the best ones
        selected_df = population_df.iloc[:num_of_indiviudals]
    # teraz musze tak ugotowac, zeby wybrac num najlepszych rodzicow 
    # i potem z nich losowo wylosowac jednego z tych wybrancow
        selected_ix = np.random.choice(selected_df.index, size=1)
        selected = selected_df.loc[selected_ix]
    return selected

**Krzyżowanie**

In [42]:
# parent1 i parent2 to juz są pojedyncze osobniki, które stają się rodzicami
def crossover(parent_1, parent_2):
    size = len(parent_1)
    # losuje losowy index osobnika, ktory bedzie podlegal krzyzowaniu
    start, end = sorted(np.random.choice(a=size, size=2, replace=False))
    
    # inicjalizuje sobie puste dzieci
    child_1 = -np.ones(size, dtype=int)
    child_2 = -np.ones(size, dtype=int)
    
    # przypisuje dziecom skrzyzowane punkty rodzica w wycinku
    child_1[start:end+1] = parent_2[start:end+1]
    child_2[start:end+1] = parent_1[start:end+1]
    
    
    # dopisuje reszte tak, zeby nie bylo powtorek
    def fill_child(child, donor_parent, segment_parent):
        for i in range(size):
            if i < start or i > end:
                candidate = donor_parent[i]
                while candidate in child:
                    # Znajdź odpowiednik w drugim rodzicu
                    pos = np.where(segment_parent == candidate)[0][0]
                    candidate = donor_parent[pos]
                child[i] = candidate
                
                
    fill_child(child_1, parent_2, parent_1)
    fill_child(child_2, parent_1, parent_2)
    
    return child_1, child_2

**Mutacja**

In [43]:
# dostaje osobnika na wejscie
def mutate(individual):
    # losuje 2 indexy 
    size = len(individual)
    first, second = np.random.choice(a=size, size=2, replace=False)
    individual[first], individual[second] = individual[second], individual[first]
    return individual

**Nowa populacja**

In [54]:
# dostaje posortowany df populacji

'''
zdaje sie, ze w generate new population cos jest zjebane, 
albo wybierany rodzicow, moze nie powinni byc kolo siebie tylko losowi kurwa 
losowi z najlepszuch 
plus sukcesja czyli populacja caly czas taki sam ma rozmiar tylko trzeba dopelniac innymi osobnikami
'''
def generate_new_population(population_df, size, succession_size, crossover_prob, mutation_prob):
    #parents = selection(population_df, 'simple', 1)
    #parents = parents.drop(['distance'], axis=1)
    elite = population_df.iloc[succession_size].copy()
    new_population = []
    while len(new_population) < size - succession_size:
        parent_1 = selection(population_df, 'simple', 1).drop(['distance'], axis=1).iloc[0]
        parent_2 = selection(population_df, 'simple', 1).drop(['distance'], axis=1).iloc[0]
        if np.random.random() <= crossover_prob:
            child_1, child_2 = crossover(parent_1, parent_2)
        else:
            child_1, child_2 = parent_1.copy(), parent_2.copy()
        if np.random.random() <= mutation_prob:
            child_1 = mutate(child_1.copy())
        if np.random.random() <= mutation_prob:
            child_2 = mutate(child_2.copy())

        new_population.append(child_1)
        new_population.append(child_2)
    new_population_df = pd.DataFrame(new_population)
    new_population_df.index.name = 'Numer osobnika'
    new_population_df.columns.names = ['Numer chromosomu']
    return new_population_df

In [59]:
# tworze sb randomowe miasta
cities_df = random_points(10, 200) # 30 roznych miast do odwiedzenia
# tworze pierwsza losowa populacje
population_df = generate_population(50, cities_df) # pierwsza populacja ma 100 osobnikow
# oceniam ta pierwsza losowa populacje
population_df = rate_population(population_df, cities_df)
print(f'5 best individuals from 1st population:\n {population_df["distance"].head()}')
new_population = rate_population(generate_new_population(population_df, size=30, succession_size=4, crossover_prob=0.85, mutation_prob=0.05), cities_df)
new_population['distance']
# new_population = generate_new_population(population_df, 8, 1, 0.05)
# rate_population(new_population, cities_df)

5 best individuals from 1st population:
 Numer osobnika
36    686.790148
31    732.116776
21    736.467253
24    752.495188
45    805.490591
Name: distance, dtype: float64


Numer osobnika
0     686.790148
23    686.790148
22    686.790148
21    686.790148
20    686.790148
19    686.790148
18    686.790148
17    686.790148
16    686.790148
15    686.790148
14    686.790148
13    686.790148
12    686.790148
11    686.790148
10    686.790148
9     686.790148
8     686.790148
7     686.790148
6     686.790148
5     686.790148
4     686.790148
3     686.790148
2     686.790148
1     686.790148
24    686.790148
25    686.790148
Name: distance, dtype: float64

In [56]:
def genetic_algorithm(num_of_cities, max_value, population_size, crossover_prob, mutation_prob):
    cities_df = random_points(num_of_cities, max_value)
    population_df = rate_population(generate_population(population_size, cities_df), cities_df)
    new_population = rate_population(generate_new_population(population_df, 2000, 4, 0.8, 0.1), cities_df)
    for i in range(300):
        new_population = rate_population(generate_new_population(new_population, 1000, 4, 0.8, 0.1), cities_df)
        mean_error = new_population['distance'].min()
        print(f'Population number {i}, min error = {mean_error}')

In [57]:
genetic_algorithm(20, 1000, 600, 0.75, 0.05)

KeyError: 0